In [2]:
import os, sys
import dotenv

import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm

sys.path.append(os.environ.get('PATH_OPENSKY'))
from user_eval import get_df_posts 
from news_outlet_eval import get_news_feed, get_posts_from_nos

dotenv.load_dotenv('.env')    
PATH_DATA = os.environ.get('PATH_OPENSKY') + os.environ.get('PATH_REL_DATA')
PATH_USER_POSTS = PATH_DATA + 'user_posts/'

PATH_RESULTS = os.environ.get('PATH_OPENSKY') + 'results/'
PATH_USER_REPLIES_NEWS = PATH_RESULTS + 'agg_user_replies_news.csv'
PATH_USER_REPLIES_NEWS_POSTS = PATH_RESULTS + 'agg_user_replies_news_posts.csv'

PATH_FEED_POSTS = PATH_DATA + 'feed_posts/'

df = pd.read_csv(PATH_USER_REPLIES_NEWS_POSTS)

/tmp/ipykernel_159850/850889407.py:23: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH_USER_REPLIES_NEWS_POSTS)


In [3]:
df

,post_id,user_id,instance,date,text,langs,like_count,reply_count,repost_count,reply_to,replied_author,thread_root,thread_root_author,repost_from,reposted_author,quotes,quoted_author,labels,sent_label,sent_score
0,10657080,284413,bsky.social,1976-05-31 14:32:20.803,let’s hope his expands ten fold 💪💪,['eng'],0,0,0,10657081.0,15717.0,10657081.0,15717.0,NaN,NaN,NaN,NaN,NaN,2.0,0.945
1,10657084,284413,bsky.social,1976-05-31 14:32:02.212,"yes, we can do this! 💪",['eng'],3,0,0,5595208.0,2486.0,5595208.0,2486.0,NaN,NaN,NaN,NaN,NaN,2.0,0.956
2,10657085,284413,bsky.social,1976-05-31 14:31:51.419,"in other words, “as long as it takes” it refer...",['eng'],0,1,0,10657086.0,12030.0,10657086.0,12030.0,NaN,NaN,NaN,NaN,NaN,1.0,0.583
3,10657096,284413,bsky.social,1976-05-31 14:31:21.410,i’ll believe it when i see the weapons in Ukra...,['eng'],0,0,0,10657097.0,15717.0,10657097.0,15717.0,NaN,NaN,NaN,NaN,NaN,1.0,0.509
4,10657117,284413,bsky.social,1976-05-31 14:30:50.922,Expect the world to become way way worse. If a...,['eng'],1,0,0,5271912.0,15717.0,5271912.0,15717.0,NaN,NaN,NaN,NaN,NaN,0.0,0.890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36816,171050451,141248,bsky.social,1976-05-31 15:04:51.347,La preuve qu'ils sont antidémocratiques. \nPer...,"['eng', 'fra']",4,0,0,5550653.0,4890.0,5550653.0,4890.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36817,105014318,92560,bsky.social,1976-05-31 15:04:40.159,What a complete abdication of good journalism ...,['eng'],0,0,0,698597.0,9.0,698597.0,9.0,NaN,NaN,NaN,NaN,NaN,0.0,0.790
36818,105014322,92560,bsky.social,1976-05-31 15:04:32.216,Trump and Biden are at par with age...exactly ...,['eng'],4,0,0,127576.0,379.0,127576.0,379.0,NaN,NaN,NaN,NaN,NaN,0.0,0.867
36819,105014501,92560,bsky.social,1976-05-31 14:30:50.808,Because doctors are literally refusing to pres...,['eng'],1,0,0,145919.0,379.0,145919.0,379.0,NaN,NaN,NaN,NaN,NaN,0.0,0.882


In [6]:
relevant_news_items = df.thread_root.unique()
df_news_feed = get_news_feed()
df_news_feed = df_news_feed[df_news_feed.id.isin(relevant_news_items)]

In [4]:
df_news_feed['news_item'] = True
df['news_item'] = False 
df = pd.concat([get_news_feed(), df])
df.loc[df.thread_root_author == 379]
df

,post_id,user_id,instance,date,text,langs,like_count,reply_count,repost_count,reply_to,...,thread_root,thread_root_author,quotes,quoted_author,labels,repost_from,reposted_author,sent_label,sent_score,news_item
0,3517724,27155,com,1976-05-30 13:16:41.125000,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
1,3517724,27155,com,1976-05-30 13:16:41.125000,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2,3517724,27155,com,1976-05-30 13:16:41.125000,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
3,73976085,12030,com,1976-05-31 15:05:40.630000,The low end consumer is still struggling\n\nht...,None,0,0,0,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
4,3517724,27155,com,1976-05-30 13:16:41.125000,📍 Welcome to 📰 News! Please hit Like ❤️ above ...,[eng],2799,46,104,NaN,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36816,171050451,141248,bsky.social,1976-05-31 15:04:51.347,La preuve qu'ils sont antidémocratiques. \nPer...,"['eng', 'fra']",4,0,0,5550653.0,...,5550653.0,4890.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
36817,105014318,92560,bsky.social,1976-05-31 15:04:40.159,What a complete abdication of good journalism ...,['eng'],0,0,0,698597.0,...,698597.0,9.0,NaN,NaN,NaN,NaN,NaN,0.0,0.790,False
36818,105014322,92560,bsky.social,1976-05-31 15:04:32.216,Trump and Biden are at par with age...exactly ...,['eng'],4,0,0,127576.0,...,127576.0,379.0,NaN,NaN,NaN,NaN,NaN,0.0,0.867,False
36819,105014501,92560,bsky.social,1976-05-31 14:30:50.808,Because doctors are literally refusing to pres...,['eng'],1,0,0,145919.0,...,145919.0,379.0,NaN,NaN,NaN,NaN,NaN,0.0,0.882,False


In [5]:
G = nx.from_pandas_edgelist(df, source='post_id', target='reply_to', edge_attr=True, create_using=nx.DiGraph)

In [14]:
import matplotlib.pyplot as plt

pos = nx.spring_layout(G, iterations=15, seed=1721)
fig, ax = plt.subplots(figsize=(15, 9))
ax.axis("off")
plot_options = {"node_size": 10, "with_labels": False, "width": 0.15}
nx.draw_networkx(G, pos=pos, ax=ax, **plot_options)

Error in callback <function _draw_all_if_interactive at 0x7fb9d4fb7be0> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

: 

: 